In [2]:
from sympy import *
init_printing()

def velocity_to_fourvelocity(v):
	vx = v[0,0]
	vy = v[1, 0]
	vz = v[2, 0]
	mod_v = sqrt(vx**2 + vy**2 + vz**2)
	gamma = 1 / sqrt(1 - mod_v**2)
	return Matrix([gamma, gamma*vx, gamma*vy, gamma*vz])

def fourvelocity_to_velocity(f):
	gamma = f[0, 0]
	return Matrix([f[1, 0] / gamma, f[2, 0] / gamma, f[3, 0] / gamma])

def scalar(u, v):
	return simplify(u[0, 0]*v[0, 0] + u[1, 0]*v[1, 0] + u[2, 0]*v[2, 0])

def modulo(v):
	return simplify(sqrt(scalar(v, v)))

def cross(u, v):
	return Matrix([
		+ u[1, 0]*v[2, 0] - u[2, 0]*v[1, 0],
		- u[0, 0]*v[2, 0] + u[2, 0]*v[0, 0],
		+ u[0, 0]*v[1, 0] - u[1, 0]*v[0, 0],
	])

def cosine(u, v):
	return scalar(u, v) / (modulo(u) * modulo(v))



def sine(u, v):
	return simplify(modulo(cross(u, v)) / (modulo(u) * modulo(v)))

def tangent_on_xy(u, v):
	assert u[2, 0] == v[2, 0] == 0
	return simplify((u[0, 0]*v[1, 0] - u[1, 0]*v[0, 0] )/ scalar(u, v))

t, x, y, z = symbols("t, x, y, z", real=True)
v, ux, uy = symbols("v, u_x, u_y", real=True)
gamma = symbols("gamma", real=True)
gamma_u = symbols("gamma_u", real=True)

n = Matrix([ 1, gamma, 0 ])

u = Matrix([ ux, uy, 0])
print("Let the velocity of the ball be u:")
u

Let the velocity of the ball be u:


⎡uₓ ⎤
⎢   ⎥
⎢u_y⎥
⎢   ⎥
⎣ 0 ⎦

In [3]:
fu = velocity_to_fourvelocity(u)
print("It's four-velocity is u^\mu =")
fu = fu.subs(1 / sqrt(1 - ux**2 - uy**2), gamma_u)
fu

It's four-velocity is u^\mu =


⎡  γᵤ  ⎤
⎢      ⎥
⎢γᵤ⋅uₓ ⎥
⎢      ⎥
⎢γᵤ⋅u_y⎥
⎢      ⎥
⎣  0   ⎦

In [4]:
L = Matrix([
	[gamma, 0, -gamma * v, 0],
	[0, 1, 0, 0],
	[-gamma*v, 0, gamma, 0],
	[0, 0, 0, 1]
])
invL = L**-1

fu_prime = simplify(L * fu)
print("In S' the four-velocity is")
fu_prime

In S' the four-velocity is


⎡γ⋅γᵤ⋅(-u_y⋅v + 1)⎤
⎢                 ⎥
⎢      γᵤ⋅uₓ      ⎥
⎢                 ⎥
⎢ γ⋅γᵤ⋅(u_y - v)  ⎥
⎢                 ⎥
⎣        0        ⎦

In [5]:
u_prime = fourvelocity_to_velocity(fu_prime)
print("It corresponds to a regular velocity (seen by S')")
u_prime

It corresponds to a regular velocity (seen by S')


⎡      uₓ      ⎤
⎢──────────────⎥
⎢γ⋅(-u_y⋅v + 1)⎥
⎢              ⎥
⎢   u_y - v    ⎥
⎢  ──────────  ⎥
⎢  -u_y⋅v + 1  ⎥
⎢              ⎥
⎣      0       ⎦

In [6]:
w_prime = Matrix([ -u_prime[1, 0], -u_prime[0, 0], u_prime[2, 0]])
print("After the ball hit the wall, it is reflected with velocity (seen by S')")
w_prime

After the ball hit the wall, it is reflected with velocity (seen by S')


⎡ -(u_y - v)   ⎤
⎢ ───────────  ⎥
⎢  -u_y⋅v + 1  ⎥
⎢              ⎥
⎢     -uₓ      ⎥
⎢──────────────⎥
⎢γ⋅(-u_y⋅v + 1)⎥
⎢              ⎥
⎣      0       ⎦

In [7]:
fw_prime = velocity_to_fourvelocity(w_prime)
print("Which correspond to a four-velocity (seen by S')")
A, B, unorm = symbols("A, B, u", positive=True)
fw_prime = fw_prime.subs(-uy*v + 1, A)
fw_prime = fw_prime.subs(gamma, 1/sqrt(1 - v**2))
fw_prime = simplify(fw_prime).expand()
fw_prime = fw_prime.subs(A, -uy*v + 1)
fw_prime = simplify(fw_prime.expand())
fw_prime = fw_prime.subs(ux**2, unorm**2 - uy**2)
fw_prime = simplify(fw_prime)
fw_prime = fw_prime.subs(unorm**2, 1-B)
fw_prime = simplify(fw_prime)
fw_prime = fw_prime.subs(B, 1-unorm**2)
fw_prime = fw_prime.subs(sqrt(1 - v**2), 1/gamma)
fw_prime = fw_prime.subs(sqrt(1 - unorm**2), 1/gamma_u)

fw_prime

Which correspond to a four-velocity (seen by S')


⎡γ⋅γᵤ⋅(-u_y⋅v + 1)⎤
⎢                 ⎥
⎢ γ⋅γᵤ⋅(-u_y + v) ⎥
⎢                 ⎥
⎢     -γᵤ⋅uₓ      ⎥
⎢                 ⎥
⎣        0        ⎦

In [8]:
fw = invL * fw_prime
print("That same four-velocity seen by S is")
fw = simplify(fw)
fw = fw.subs(1 - v**2, 1/gamma**2)
fw

That same four-velocity seen by S is


⎡-γ⋅γᵤ⋅(γ⋅(u_y⋅v - 1) + uₓ⋅v)⎤
⎢                            ⎥
⎢      γ⋅γᵤ⋅(-u_y + v)       ⎥
⎢                            ⎥
⎢-γ⋅γᵤ⋅(γ⋅v⋅(u_y⋅v - 1) + uₓ)⎥
⎢                            ⎥
⎣             0              ⎦

In [9]:

w = fourvelocity_to_velocity(fw)
w = simplify(w)
print("And it correspond to a velocity")
w

And it correspond to a velocity


⎡      u_y - v       ⎤
⎢────────────────────⎥
⎢γ⋅(u_y⋅v - 1) + uₓ⋅v⎥
⎢                    ⎥
⎢γ⋅v⋅(u_y⋅v - 1) + uₓ⎥
⎢────────────────────⎥
⎢γ⋅(u_y⋅v - 1) + uₓ⋅v⎥
⎢                    ⎥
⎣         0          ⎦

In [20]:
tan_in = simplify(tangent_on_xy(-u, n))
tan_out = simplify(tangent_on_xy(w, n))
ratio = simplify(tan_out / tan_in).expand()
ratio = ratio.subs(v**2, 1 - 1/gamma**2)
ratio = simplify(ratio)
# ratio.factor(1 - v**2)
print(latex(ratio))

\frac{\gamma u_{y} + u_{x}}{\gamma \left(- \gamma^{2} u_{y} + \gamma^{2} v - \gamma u_{x} + v\right)}
